In [33]:

import pandas as pd
import numpy as np
import os
from datetime import date

In [34]:
input_file_name = "input_09_06_24.csv" 
input_file_path = os.path.join(os.getcwd(), 'inputs', input_file_name)
output_file_path = os.path.join(os.getcwd(), f'outputs/output_{date.today().strftime("%m_%d_%y")}.csv')

In [43]:
os.getcwd()

'c:\\Users\\Lenovo ThinkPad\\Documents\\Golf\\Code'

In [35]:
df = pd.read_csv(input_file_path,na_values="--",skiprows=11)

grouped_data = df.groupby('Order number',group_keys=False)

clone_df = df.drop(df[df['Type'] != 'Order'].index)

clean_df = clone_df.drop( ["Type", "Net amount", "Payout currency", "Payout ID", "Payout status",
    "Item ID", "Custom label", "Quantity", "Final value fee – fixed", "Final value fee – variable", "Regulatory operating fee",
    "Very high 'item not as described' fee", "Below standard performance fee",
    "International fee", "Gross transaction amount", "Transaction currency",
    "Exchange rate", "Reference ID",  "Description"],axis=1)


In [36]:

def formatFees(group):
    indices_dict = {
        "Order": group.index[group['Type'] == 'Order'].tolist(),
        "Ad fee" : group.index[group['Description'] == 'Ad Fee Standard '].tolist(),
        "Refund" : group.index[group['Type'] == 'Refund'].tolist(),
        "Refund postage" : group.index[group['Type'] == 'Postage label'].tolist(),
        "Hold" : group.index[group['Type'] == 'Hold'].tolist()
        }
    
    gross = df['Gross transaction amount'].iloc[indices_dict["Order"]].dropna() \
        + df['Gross transaction amount'].iloc[indices_dict["Refund"]].dropna().sum()


    ebay_fees = df['Final value fee – fixed'].iloc[indices_dict["Order"]].dropna().sum() \
        + df['Final value fee – variable'].iloc[indices_dict["Order"]].dropna().sum() \
        + df['Final value fee – variable'].iloc[indices_dict["Refund"]].dropna().sum() \
        + df['Regulatory operating fee'].iloc[indices_dict["Order"]].dropna().sum()
    
    ad_fees = df['Gross transaction amount'].iloc[indices_dict["Ad fee"]].dropna().sum()

    postage_label = df['Gross transaction amount'].iloc[indices_dict["Refund postage"]].dropna().sum()

    df["Custom label"] = df['Custom label'].str.replace(" ","")

    number_balls = pd.DataFrame(np.zeros(len(grouped_data)))
    number_balls = df['Custom label'].str.extract(r'-(\d+)')[0].astype(float)





#checks to see if item was refunded
    if indices_dict.get("Refund"):
        if indices_dict.get("Hold"): #check for return
            ad_fees=0
        else: #action taken if only partial refund
            refunded_amount = df['Gross transaction amount'].iloc[indices_dict["Refund"]].dropna()
            ad_fees += ad_fees*float(refunded_amount) / df['Gross transaction amount'].iloc[indices_dict["Order"]].dropna()

    
    net_sale = gross + ebay_fees + ad_fees + postage_label
    new_df = pd.DataFrame({'Order number': group['Order number'],'Number of balls': number_balls.fillna(0), 'Gross sale': gross, 'Ebay fee': ebay_fees, 'Ad fee': ad_fees, 'Postage': 0,'Return postage': postage_label, 'Net sale': "" }).dropna()
    return new_df




In [37]:
formatted_df = grouped_data.apply(formatFees)
sales_df = pd.merge(clean_df,formatted_df)


sales_df['Transaction creation date'] = pd.to_datetime(sales_df["Transaction creation date"],format= "%d %b %Y")
sales_df.sort_values(by='Transaction creation date', inplace=True)
sales_df = sales_df.drop_duplicates(subset=["Order number", "Buyer username"])


In [38]:

output_file = output_file_path 
print(output_file)

try:
    sales_df.to_csv(output_file, index=False)
    x = pd.read_csv(output_file)
    print(x)
    print("output succesfful")
    
except FileNotFoundError as e:
    print(f"File not found error: {e}")
except PermissionError as e:
    print(f"Permission error: {e}. The file might be open in another program. Please close it and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


# os.startfile(f'"C:/Users/Lenovo ThinkPad/Documents/Golf/Code/{output_file}"')
# os.startfile(f'"C:/Users/Lenovo ThinkPad/Documents/Golf/Code/{output_file}"')
# input()



c:/outputs/output_07_22_24.csv
An error occurred: Cannot save file into a non-existent directory: 'c:\outputs'


In [40]:
output_file_path

'c:/outputs/output_07_22_24.csv'